Setup

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import torch
import torch.nn
import torch.optim as optim
import sqlite3
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize

#from helper import
from DataPreprocessing import DataPreprocesser

In [2]:
connection = sqlite3.connect('dota2.db')
cursor = connection.cursor()
MyProcesser = DataPreprocesser(connection, cursor)

Load Data

In [3]:
DataPreprocesser.to_dataframes(MyProcesser)  # Load the data into dataframes

data = MyProcesser.merge_data()

y = data.loc[:, 'radiant_win']
X = data.drop(columns=['radiant_win'])

Total number of matches available: 8807
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (1, 21)
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (2, 41)
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (3, 61)
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (3, 81)
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (4, 101)
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (5, 121)
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (6, 141)
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (7, 161)
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (8, 181)
The new player to insert is of shape: (1, 20)
Updated temp_players is now of shape: (9, 201)
Shape of data is now: (9, 201)
The

KeyboardInterrupt: 

Data Setup

Model Setup

Train Model

Evaluate Model

Performance Analysis

Save Data